# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd



In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')
df

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A


In [3]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [7]:
df_filtered = df[(df['Total Claim Amount'] > 1_000 & (df['Response'] == 'Yes'))]
df_filtered

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A


In [9]:
df_groupered = df_filtered.groupby('Total Claim Amount').sum()
df_groupered

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size,Vehicle Type
Total Claim Amount,,,,,,,,,,,,,,,,,,,,,
0.099007,3663,EY74093,Nevada,5004.135361,No,Basic,High School or Below,2/21/11,Employed,M,...,93,0.0,8,Corporate Auto,Corporate L1,Offer4,Agent,Four-Door Car,Medsize,A
0.382107,6869,PN21042PN21042,ArizonaArizona,9077.695636,NoNo,BasicBasic,BachelorBachelor,2/17/112/17/11,EmployedEmployed,MM,...,124,0.0,2,Personal AutoPersonal Auto,Personal L3Personal L3,Offer1Offer1,Call CenterCall Center,Sports CarSports Car,MedsizeMedsize,0
0.423310,13169,IS30186IS30186,CaliforniaCalifornia,12372.995662,NoNo,ExtendedExtended,CollegeCollege,1/31/111/31/11,EmployedEmployed,FF,...,22,0.0,6,Personal AutoPersonal Auto,Personal L2Personal L2,Offer2Offer2,AgentAgent,Four-Door Car,Medsize,A
0.517753,7567,HG65722,Oregon,12819.102890,No,Premium,Doctor,1/25/11,Disabled,F,...,4,3.0,3,Personal Auto,Personal L2,Offer1,Branch,SUV,Medsize,0
0.769185,16457,WW74234WW74234,Arizona,7938.866354,No,BasicBasic,CollegeCollege,2/5/112/5/11,EmployedEmployed,FF,...,58,0.0,2,Personal AutoPersonal Auto,Personal L3Personal L3,Offer1Offer1,AgentAgent,SUVSUV,MedsizeMedsize,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345.413441,641,FO35655,Oregon,8672.229719,No,Extended,Bachelor,1/14/11,Unemployed,M,...,83,0.0,1,Corporate Auto,Corporate L3,Offer1,Branch,Luxury SUV,Medsize,A
2452.894264,1319,UZ14076,California,32104.699570,No,Premium,High School or Below,2/16/11,Employed,F,...,18,1.0,2,Corporate Auto,Corporate L3,Offer3,Agent,Luxury SUV,Large,0
2552.343856,17957,IV94496IV94496,NevadaNevada,86434.432220,NoNo,ExtendedExtended,High School or BelowHigh School or Below,1/27/111/27/11,EmployedEmployed,MM,...,198,0.0,4,Personal AutoPersonal Auto,Personal L3Personal L3,Offer1Offer1,AgentAgent,Luxury SUV,Medsize,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [10]:
df02_average = df[(df['Response'] == 'Yes')].groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean()
df02_average

Policy Type     Gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: Total Claim Amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [18]:
df03 = df['State'].value_counts()

df03 = df03[df03 > 500]

df03

State
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

KeyError: ('State', 'Month')

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [24]:
df07 = pd.melt(df[df['Response'] == 'Yes'], id_vars=['Sales Channel'], value_vars=['Response'])
df07 = df07['Sales Channel'].value_counts()
df07

Sales Channel
Agent          742
Branch         326
Call Center    221
Web            177
Name: count, dtype: int64

In [5]:
# Convertir la columna de fechas al tipo datetime 
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'], format='%m/%d/%y') 

# Agregar una columna adicional para el mes y el año 
df['Month'] = df['Effective To Date'].dt.to_period('M')

df_grouped_by_state_and_month = df.groupby(['State', 'Month'])['Number of Policies'].sum()
df_grouped_by_state_and_month

State       Month  
Arizona     2011-01    3052
            2011-02    2864
California  2011-01    5673
            2011-02    4929
Nevada      2011-01    1493
            2011-02    1278
Oregon      2011-01    4697
            2011-02    3969
Washington  2011-01    1358
            2011-02    1225
Name: Number of Policies, dtype: int64

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here